# Scraping rottentomatoes

In [1]:
import re
import numpy as np
import pandas as pd
import json
from random import randint, uniform,random
import time

In [2]:
contador = randint(20,50)
contador

42

In [3]:
datos_excel= pd.read_excel("C:/Users/ASUS/Downloads/movies_filter.xlsx")
datos_excel.head()

,budget,company,country,director,genre,gross,name,rating,day_released,month_released,year_released,released,runtime,score,star,votes,writer,year,hashtag
0,100000000.0,Red Granite Pictures,USA,Martin Scorsese,Biography,116900694.0,The Wolf of Wall Street,R,25,12,2013,2013-12-25,180,8.2,Leonardo DiCaprio,895552,Terence Winter,2013,#TheWolfofWallStreet
1,20000000.0,New Line Cinema,USA,James Wan,Horror,137400141.0,The Conjuring,R,19,7,2013,2013-07-19,112,7.5,Patrick Wilson,342870,Chad Hayes,2013,#TheConjuring
2,46000000.0,Alcon Entertainment,USA,Denis Villeneuve,Crime,61002302.0,Prisoners,R,20,9,2013,2013-09-20,153,8.1,Hugh Jackman,449489,Aaron Guzikowski,2013,#Prisoners
3,150000000.0,Walt Disney Animation Studios,USA,Chris Buck,Animation,400738009.0,Frozen,PG,27,11,2013,2013-11-27,102,7.5,Kristen Bell,464149,Jennifer Lee,2013,#Frozen
4,75000000.0,Summit Entertainment,USA,Louis Leterrier,Crime,117723989.0,Now You See Me,PG-13,31,5,2013,2013-05-31,115,7.3,Jesse Eisenberg,505432,Ed Solomon,2013,#NowYouSeeMe


In [4]:
def pre_procesado(texto):
    texto=str(texto)
    texto = texto.lower()
    #texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = re.sub(r"[\s]+", "_", texto) 
    #texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    #texto = " ".join([spanishStemmer.stem(palabra) for palabra in texto.split() ])
    return texto

In [5]:
datos_proc=datos_excel

In [6]:
datos_proc["name_roten"]=  datos_proc["name"].apply(lambda texto: pre_procesado(texto))
datos_proc.head()

,budget,company,country,director,genre,gross,name,rating,day_released,month_released,year_released,released,runtime,score,star,votes,writer,year,hashtag,name_roten
0,100000000.0,Red Granite Pictures,USA,Martin Scorsese,Biography,116900694.0,The Wolf of Wall Street,R,25,12,2013,2013-12-25,180,8.2,Leonardo DiCaprio,895552,Terence Winter,2013,#TheWolfofWallStreet,the_wolf_of_wall_street
1,20000000.0,New Line Cinema,USA,James Wan,Horror,137400141.0,The Conjuring,R,19,7,2013,2013-07-19,112,7.5,Patrick Wilson,342870,Chad Hayes,2013,#TheConjuring,the_conjuring
2,46000000.0,Alcon Entertainment,USA,Denis Villeneuve,Crime,61002302.0,Prisoners,R,20,9,2013,2013-09-20,153,8.1,Hugh Jackman,449489,Aaron Guzikowski,2013,#Prisoners,prisoners
3,150000000.0,Walt Disney Animation Studios,USA,Chris Buck,Animation,400738009.0,Frozen,PG,27,11,2013,2013-11-27,102,7.5,Kristen Bell,464149,Jennifer Lee,2013,#Frozen,frozen
4,75000000.0,Summit Entertainment,USA,Louis Leterrier,Crime,117723989.0,Now You See Me,PG-13,31,5,2013,2013-05-31,115,7.3,Jesse Eisenberg,505432,Ed Solomon,2013,#NowYouSeeMe,now_you_see_me


In [7]:
datos_proc["calificacion1"]= ""
datos_proc["calificacion2"] = ""
datos_proc["synopsis"]=""

In [8]:
# Encabezado, no título
import urllib.request
import bs4 as bs
import numpy as np
url = "https://www.rottentomatoes.com/m/the_conjuring"

In [9]:
def busq_html(url,contador):
    request = urllib.request.urlopen(url)
    fuente = request.read()
    request.close()
    arreglo_final_dato=encontrar_dato(fuente,contador)
    return arreglo_final_dato

In [10]:
def pre_procesado_2ejec(texto):
    texto = texto.lower()
    #texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = re.sub(r"[^A-z 0-9 % ]+", " ", texto) 
    texto = texto.strip()
    #texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    #texto = " ".join([spanishStemmer.stem(palabra) for palabra in texto.split() ])
    return texto

### Realizamos el scraping para traer los datos de synopsis, calificacion de los criticos y calificacion del publico

In [11]:
def encontrar_dato(fuente,contador):
    arreglo_link={}
    contador_link=-1
    pagina = bs.BeautifulSoup(fuente, "html.parser")
    validador=pre_procesado_2ejec(pagina.find(class_="mop-ratings-wrap__text--small").text)
    print(validador)
    if validador == "tomatometer  not yet available":
        datos_depurados["synopsis"][contador] = "Sin Datos"
    else :
        calificaciones =pagina.find_all(class_="mop-ratings-wrap__percentage")
        calificacion1=calificaciones[0].text
        calificacion2=calificaciones[1].text
        #calificacion2=pagina.find(class_="mop-ratings-wrap__percentage").text
        synopsis=pagina.find(class_="movie_synopsis").text  
        datos_depurados["calificacion1"][contador] = pre_procesado_2ejec(calificacion1)
        datos_depurados["calificacion2"][contador] = pre_procesado_2ejec(calificacion2)
        datos_depurados["synopsis"][contador] = pre_procesado_2ejec(synopsis)

In [12]:
contador = 0
datos_proc["ruta"]=""
for filas in datos_proc["name_roten"] :
    roten = datos_proc["name_roten"][contador]
    url = "https://www.rottentomatoes.com/m/"+roten
    datos_proc["ruta"][contador] = url.strip()
    print(url)
    #prueba_datos=busq_html(url,contador)
    contador +=1
    #contador = randint(20,50)
    #time.sleep(contador) 
prueba_datos

<ipython-input-12-735110e7d49b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_proc["ruta"][contador] = url.strip()


https://www.rottentomatoes.com/m/the_wolf_of_wall_street
https://www.rottentomatoes.com/m/the_conjuring
https://www.rottentomatoes.com/m/prisoners
https://www.rottentomatoes.com/m/frozen
https://www.rottentomatoes.com/m/now_you_see_me
https://www.rottentomatoes.com/m/her
https://www.rottentomatoes.com/m/man_of_steel
https://www.rottentomatoes.com/m/we're_the_millers
https://www.rottentomatoes.com/m/blue_is_the_warmest_color
https://www.rottentomatoes.com/m/pacific_rim
https://www.rottentomatoes.com/m/the_hunger_games:_catching_fire
https://www.rottentomatoes.com/m/american_hustle
https://www.rottentomatoes.com/m/gravity
https://www.rottentomatoes.com/m/12_years_a_slave
https://www.rottentomatoes.com/m/star_trek_into_darkness
https://www.rottentomatoes.com/m/olympus_has_fallen
https://www.rottentomatoes.com/m/oblivion
https://www.rottentomatoes.com/m/dallas_buyers_club
https://www.rottentomatoes.com/m/about_time
https://www.rottentomatoes.com/m/world_war_z
https://www.rottentomatoes.com

NameError: name 'prueba_datos' is not defined

In [ ]:
#from pandas import ExcelWriter
#datos_proc
#writer = ExcelWriter('C:/Users/ASUS/Downloads/rotten.xlsx')
#datos_proc.to_excel(writer, 'Hoja de datos', index=False)
#writer.save()

In [15]:
datos_depurados=pd.read_excel("C:/Users/ASUS/Downloads/rotten.xlsx")
contador = 0
validador_fila = 666
for filas in datos_depurados["name_roten"] :
    if  contador >= validador_fila and  contador <= 1000 :
        roten = datos_depurados["name_roten"][contador]
        url = datos_depurados["ruta"][contador].strip()
        #datos_proc["ruta"][contador] = url.strip()
        print(f"URL :{url} ...contador:{contador}")
        prueba_datos=busq_html(url,contador)
        contador +=1
        aleatorio = randint(1,5)
        time.sleep(aleatorio) 
    else :
        contador +=1

URL :https://www.rottentomatoes.com/m/31 ...contador:666
total count


<ipython-input-11-3c6d5003ea42>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_depurados["calificacion1"][contador] = pre_procesado_2ejec(calificacion1)
<ipython-input-11-3c6d5003ea42>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_depurados["calificacion2"][contador] = pre_procesado_2ejec(calificacion2)
<ipython-input-11-3c6d5003ea42>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_depurados["synopsis"][conta

URL :https://www.rottentomatoes.com/m/the_comedian ...contador:667
total count
URL :https://www.rottentomatoes.com/m/billy_lynns_long_halftime_walk ...contador:668
total count
URL :https://www.rottentomatoes.com/m/petes_dragon_2016 ...contador:669
total count
URL :https://www.rottentomatoes.com/m/money_monster ...contador:670
total count
URL :https://www.rottentomatoes.com/m/free_state_of_jones ...contador:671
total count
URL :https://www.rottentomatoes.com/m/wakefield ...contador:672
total count
URL :https://www.rottentomatoes.com/m/mothers_day_2016 ...contador:673
total count
URL :https://www.rottentomatoes.com/m/rules_dont_apply ...contador:674
total count
URL :https://www.rottentomatoes.com/m/dead_awake ...contador:675
tomatometer  not yet available


<ipython-input-11-3c6d5003ea42>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_depurados["synopsis"][contador] = "Sin Datos"


URL :https://www.rottentomatoes.com/m/loving_2016 ...contador:676
total count
URL :https://www.rottentomatoes.com/m/lowriders_2017 ...contador:677
total count
URL :https://www.rottentomatoes.com/m/tyler_perrys_boo_a_madea_halloween ...contador:678
total count
URL :https://www.rottentomatoes.com/m/in_a_valley_of_violence ...contador:679
total count
URL :https://www.rottentomatoes.com/m/the_forest ...contador:680
tomatometer  not yet available
URL :https://www.rottentomatoes.com/m/whiskey_tango_foxtrot ...contador:681
total count
URL :https://www.rottentomatoes.com/m/max_steel ...contador:682
total count
URL :https://www.rottentomatoes.com/m/busters_mal_heart ...contador:683
total count
URL :https://www.rottentomatoes.com/m/i_daniel_blake ...contador:684
total count
URL :https://www.rottentomatoes.com/m/the_red_turtle ...contador:685
total count
URL :https://www.rottentomatoes.com/m/a_street_cat_named_bob ...contador:686
total count
URL :https://www.rottentomatoes.com/m/genius_2016 ...co

In [16]:
from pandas import ExcelWriter
datos_depurados
writer = ExcelWriter('C:/Users/ASUS/Downloads/rotten_inic13.xlsx')
datos_depurados.to_excel(writer, 'Hoja de datos', index=False)
writer.save()
datos_depurados

,budget,company,country,director,genre,gross,name,rating,day_released,month_released,...,star,votes,writer,year,hashtag,name_roten,calificacion1,calificacion2,synopsis,ruta
0,100000000,Red Granite Pictures,USA,Martin Scorsese,Biography,116900694,The Wolf of Wall Street,R,25,12,...,Leonardo DiCaprio,895552,Terence Winter,2013,#TheWolfofWallStreet,the_wolf_of_wall_street,NaN,NaN,Sin Datos,https://www.rottentomatoes.com/m/the_wolf_of_w...
1,20000000,New Line Cinema,USA,James Wan,Horror,137400141,The Conjuring,R,19,7,...,Patrick Wilson,342870,Chad Hayes,2013,#TheConjuring,the_conjuring,NaN,NaN,NaN,https://www.rottentomatoes.com/m/the_conjuring
2,46000000,Alcon Entertainment,USA,Denis Villeneuve,Crime,61002302,Prisoners,R,20,9,...,Hugh Jackman,449489,Aaron Guzikowski,2013,#Prisoners,prisoners,NaN,NaN,NaN,https://www.rottentomatoes.com/m/prisoners_2013
3,150000000,Walt Disney Animation Studios,USA,Chris Buck,Animation,400738009,Frozen,PG,27,11,...,Kristen Bell,464149,Jennifer Lee,2013,#Frozen,frozen,NaN,NaN,NaN,https://www.rottentomatoes.com/m/frozen
4,75000000,Summit Entertainment,USA,Louis Leterrier,Crime,117723989,Now You See Me,PG-13,31,5,...,Jesse Eisenberg,505432,Ed Solomon,2013,#NowYouSeeMe,now_you_see_me,NaN,NaN,NaN,https://www.rottentomatoes.com/m/now_you_see_me
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,0,Fox Searchlight Pictures,UK,Mandie Fletcher,Comedy,4750497,Absolutely Fabulous: The Movie,R,22,7,...,Jennifer Saunders,9161,Jennifer Saunders,2016,#AbsolutelyFabulous:TheMovie,absolutely_fabulous:_the_movie,59%,41%,publicist edina monsoon and best friend patsy ...,https://www.rottentomatoes.com/m/absolutely_fa...
719,0,Siempre Viva Productions,USA,Paul Duddridge,Drama,28368,Mothers and Daughters,PG-13,6,5,...,Selma Blair,1959,Paige Cameron,2016,#MothersandDaughters,mothers_and_daughters,18%,28%,a pregnant photographer selma blair captures...,https://www.rottentomatoes.com/m/mothers_and_d...
720,3500000,Warner Bros. Animation,USA,Sam Liu,Animation,3775000,Batman: The Killing Joke,R,25,7,...,Kevin Conroy,36333,Brian Azzarello,2016,#Batman:TheKillingJoke,batman:_the_killing_joke,40%,46%,batman kevin conroy must save commissioner g...,https://www.rottentomatoes.com/m/batman_the_ki...
721,0,Borderline Presents,USA,Nicolas Pesce,Drama,25981,The Eyes of My Mother,R,2,12,...,Kika Magalhães,6947,Nicolas Pesce,2016,#TheEyesofMyMother,the_eyes_of_my_mother,78%,57%,francisca kika magalhaes has been unfazed by...,https://www.rottentomatoes.com/m/the_eyes_of_m...
